In [1]:
import sympy as sym
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from scipy.special import jv, jn_zeros
from sympy import Derivative as D, Function as F

import IPython.display as dis
from IPython.display import HTML

%matplotlib inline
plt.rcParams['animation.html'] = 'jshtml'

Riešenie vlnovej rovnice pre kruhovú membránu
---------------------------------------------------------------------

Vlnová rovnica v polárnych súradniciach:

In [2]:
r, theta, t, r1, c, u0 = sym.symbols("r theta t r1 c u0", positive = True)
u = F('u')(r, theta, t)

# vlnova rovnica
ls = D(u, r, 2) + (1/r)*D(u, r, 1) + (1/r**2)*D(u, theta, 2)
rs = (1/c**2)*D(u, t, 2)
eq = sym.Eq(ls, rs)
eq

Eq(Derivative(u(r, theta, t), (r, 2)) + Derivative(u(r, theta, t), r)/r + Derivative(u(r, theta, t), (theta, 2))/r**2, Derivative(u(r, theta, t), (t, 2))/c**2)

Riešime pomocou rozdelenia funkcie popisujúcej výchylku na tri funkcie nezávislých premenných: $u(r,\theta,t) = R(r)H(\theta)T(t)$. Knižnica ``sympy`` ponúka funkciu ``pde_separate_mul``, tá však nefunguje pre funkcie viac ako dvoch premenných, preto je potrebné postupovať "manuálne".

In [3]:
# oddelenie premennych
R = F('R')
H = F('H')
T = F('T')

# pde_separate_mul nefunguje, nutne rucne oddelenie

eq2 = eq.subs(u, R(r)*H(theta)*T(t)).doit()
eq2

Eq(H(theta)*T(t)*Derivative(R(r), (r, 2)) + H(theta)*T(t)*Derivative(R(r), r)/r + R(r)*T(t)*Derivative(H(theta), (theta, 2))/r**2, H(theta)*R(r)*Derivative(T(t), (t, 2))/c**2)

In [4]:
eq3 = sym.Eq(sym.expand_mul(eq2.lhs / (R(r)*H(theta)*T(t))), sym.expand_mul(eq2.rhs / (R(r)*H(theta)*T(t))))
eq3

Eq(Derivative(R(r), (r, 2))/R(r) + Derivative(R(r), r)/(r*R(r)) + Derivative(H(theta), (theta, 2))/(r**2*H(theta)), Derivative(T(t), (t, 2))/(c**2*T(t)))

Získali sme rovnicu s oddelenými premennými. Obe jej strany porovnáme s konštantou, ktorú zvolíme ako $-\lambda^2$. Získame prvú diferenciálnu rovnicu s funkciou času:

In [5]:
# lhs = rhs = K = -lambda**2

lambda_, m = sym.symbols("lambda m", positive = True)
eq4 = sym.Eq(eq3.rhs, -lambda_**2)
eq4

Eq(Derivative(T(t), (t, 2))/(c**2*T(t)), -lambda**2)

In [6]:
# obe strany vynasobime r**2
eq5 = sym.Eq((eq3.lhs * r**2).simplify(), (-lambda_**2 * r**2).simplify())
eq5 = sym.Eq(eq5.lhs - eq5.rhs, 0)
eq5

Eq(lambda**2*r**2 + r**2*Derivative(R(r), (r, 2))/R(r) + r*Derivative(R(r), r)/R(r) + Derivative(H(theta), (theta, 2))/H(theta), 0)

Porovnáme časti rovnice obsahujúce jednotlivé premenné s konštantou, ktorú zvolíme ako $m^2$. Získame tak zvyšné dve diferenciálne rovnice.

In [7]:
# lhs = rhs = L = m**2
sep_R, sep_H = eq5.lhs.as_independent(theta)
eq6 = sym.Eq(sep_R, m**2)
eq6

Eq(lambda**2*r**2 + r**2*Derivative(R(r), (r, 2))/R(r) + r*Derivative(R(r), r)/R(r), m**2)

In [8]:
eq7 = sym.Eq(-sep_H, m**2)
eq7

Eq(-Derivative(H(theta), (theta, 2))/H(theta), m**2)

Riešime jednotlivé diferenciálne rovnice pomocou funkcie ``dsolve``:

In [9]:
K1, K2, K3, K4 = sym.symbols("K1 K2 K3 K4")

In [10]:
H_sol = sym.dsolve(eq7, H(theta))
const_H = list(H_sol.free_symbols - eq7.free_symbols)

# nahradime vystupne konstanty vlastnymi konstantami, pricom K1 prislucha funkcii cos
if (H_sol.subs(const_H[0], 0).has(sym.sin)):
    H_sol = H_sol.subs([(const_H[1], K2), (const_H[0], K1)])
else:
    H_sol = H_sol.subs([(const_H[0], K2), (const_H[1], K1)])

H_sol

Eq(H(theta), K1*cos(m*theta) + K2*sin(m*theta))

In [11]:
R_sol = sym.dsolve(eq6, R(r))
R_sol

Eq(R(r), C1*besselj(m, lambda*r) + C2*bessely(m, lambda*r))

Aby $R(0)$ bolo konečné, musí byť koeficient besselovej funkcie $Y_n$ nulový. Koeficient funkcie $J_n$ môže byť obsiahnutý v ďalších konštantách vo výslednej funkcii, preto zvolíme jeho hodnotu $1$.

In [12]:
# chceme, aby v rieseni zostala iba funkcia J_n
# je potrebne zistit, ktora konstanta prislucha ktorej funkcii:

const_R = list(R_sol.free_symbols - eq6.free_symbols)
if (R_sol.subs(const_R[0], 0).has(sym.besselj)):
    R_sol = R_sol.subs([(const_R[1], 1), (const_R[0], 0)])
else:
    R_sol = R_sol.subs([(const_R[0], 1), (const_R[1], 0)])

R_sol

Eq(R(r), besselj(m, lambda*r))

In [13]:
T_sol = sym.dsolve(eq4, T(t))#, ics = {T(0): u0, T(t).diff(t).subs(t, 0): 0})
const_T = list(T_sol.free_symbols - eq4.free_symbols)
if (T_sol.subs(const_T[0], 0).has(sym.sin)):
    T_sol = T_sol.subs([(const_T[1], K4), (const_T[0], K3)])
else:
    T_sol = T_sol.subs([(const_T[0], K4), (const_T[1], K3)])

T_sol = T_sol.simplify()
T_sol

Eq(T(t), K3*cos(c*lambda*t) + K4*sin(c*lambda*t))

Vynásobíme riešenia jednotlivých diferenicálnych rovníc a získame tak všeobecné riešenie vlnovej rovnice. Konštanty závisia od počiatočných podmienok.

In [14]:
u_sol = R_sol.rhs*H_sol.rhs*T_sol.rhs
u_sol

(K1*cos(m*theta) + K2*sin(m*theta))*(K3*cos(c*lambda*t) + K4*sin(c*lambda*t))*besselj(m, lambda*r)

Overenie platnosti riešenia dosadením do vlnovej rovnice:

In [15]:
ls = u_sol.diff(r, r) + (1/r)*u_sol.diff(r) + (1/r**2)*u_sol.diff(theta, theta)
rs = (1/c**2)*u_sol.diff(t, t)

if rs.equals(ls):
    print("Riešenie vlnovej rovnice je platné.")
else:
    print("Riešenie vlnovej rovnice nie je platné.")

Riešenie vlnovej rovnice je platné.


Aby bola splnená okrajová podmienka $u(r_1, \theta, t) = 0$, kde $r_1$ je polomer membrány (na okrajoch je výchylka vždy nulová), musí platiť:

In [16]:
# pociatocna podmienka u(r1, theta, t) = 0
sym.Eq(R_sol.rhs.subs(r, r1), 0)

Eq(besselj(m, lambda*r1), 0)

$\lambda$ je koreňom besselovej funkcie $J_m \implies$ nekonečne mnoho kladných koreňov $\lambda_{mn}$

$J_m(k_{mn}) = 0 \implies \lambda_{mn} = \frac{k_{mn}}{r_1}$, kde $k_{mn}$ je $n$-tou nulou besselovej funkcie $J_m$.

Použijeme substitúciu:

In [17]:
k_mn = sym.symbols("k_mn")
lambda_mn = k_mn/r1
u_sol = u_sol.subs(lambda_, lambda_mn)
u_sol

(K1*cos(m*theta) + K2*sin(m*theta))*(K3*cos(c*k_mn*t/r1) + K4*sin(c*k_mn*t/r1))*besselj(m, k_mn*r/r1)

Platí $a\cos\phi + b\sin\phi = \sqrt{a^2 + b^2} \cos(\phi + \phi_0)$. Ak nám ide len o kvalitatívny popis jednotlivých rezonančných módov, môžeme fázový posun zanedbať. Preto zvolíme $K_2 = K_4 = 0$. Pri kvalitatívnom popise berieme výchylku ako bezrozmernú veličinu, dosadíme tak za zvyšné konštanty $K_1 = K_3 = 1$. Dostaneme tak zjednodušenú funkciu popisujúcu konkrétny mód:

In [18]:
u_sol_q = u_sol.subs([(K1, 1), (K2, 0), (K3, 1), (K4, 0)])
u_sol_q

cos(m*theta)*cos(c*k_mn*t/r1)*besselj(m, k_mn*r/r1)

Dosadíme ľubovolné hodnoty veličín iba za účelom vizualizácie (nezodpovedajú reálnym parametrom):

In [19]:
# povrchove napatie
P0 = 2.5 # N*m-1
# plosna hustota
sigma1 = 10 # kg*m-2
# rychlost sirenia vlny
c1 = np.sqrt(P0/sigma1) # m*s-1
# polomer membrany
r2 = 0.35 # m

In [20]:
# mod 0, 1
k11 = jn_zeros(0, 1)
u11 = u_sol_q.subs([(k_mn, k11[0]), (c, c1), (r1, r2), (m, 0)])
u11

cos(3.43546508242253*t)*besselj(0, 6.87093016484507*r)

Animované zobrazenie rezonančných módov membrány $u_{mn}(x, y, t)$:
-----------------------------------------------------------------------------------

In [21]:
plt.rcParams['figure.figsize'] = [12, 8]

m_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    description='m = '
)
n_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    description='n = '
)

button = widgets.Button(description='Aktualizovat')

def update_display():
    dis.clear_output(wait=True)
    dis.display(m_slider)
    dis.display(n_slider)
    dis.display(button)
    print("f =", freq, "Hz")

def update(m1, n1):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    
    # funkcia vychylky pre konkretny mod
    k = jn_zeros(m1, n1)[n1 - 1]
    u_mn = u_sol_q.subs([(k_mn, k), (c, c1), (r1, r2), (m, m1)])
    f = sym.lambdify([r, theta, t], u_mn, modules=[{'besselj': jv}, 'scipy'])
    
    # vytvorenie os, mesh
    r_space = np.linspace(0, r2, 50)
    p_space = np.linspace(0, 2*np.pi, 50)
    R_mesh, P_mesh = np.meshgrid(r_space, p_space)
    
    T = 2*np.pi/(c1*k/r2) # perioda
    frn = 15 # pocet snimkov
    fps = 10 # pocet snimkov za sekundu
    
    global freq
    freq = 1/T
    
    U = np.zeros((50, 50, frn))
    for i in range(frn):
        U[:,:,i] = f(R_mesh, P_mesh, ((T/frn)*i))
    

    # prevedenie do kartezianskeho suradnicoveho systemu
    X, Y = R_mesh*np.cos(P_mesh), R_mesh*np.sin(P_mesh)
    
    def update_plot(frame_number, zarray, plot):
        plot[0].remove()
        plot[0] = ax.plot_surface(X, Y, U[:,:,frame_number], cmap=plt.cm.viridis)
    
    # ohranicenie osi vychylky, popisanie os
    ax.set_zlim(-1.2, 1.2)
    ax.set_xlabel(r'$x$', fontsize=14)
    ax.set_ylabel(r'$y$', fontsize=14)
    ax.set_zlabel(r'$u(x, y, t)$', fontsize=14)

    plot = [ax.plot_surface(X, Y, U[:,:,0], color='0.75', rstride=1, cstride=1)]
    ani = animation.FuncAnimation(fig, update_plot, frames=frn, fargs=(U, plot), interval=1000/fps)
    update_display()
    dis.display(ani)
    plt.close()
    
def on_button_clicked(_):
    update(m_slider.value, n_slider.value)
button.on_click(on_button_clicked)


update(m_slider.value, n_slider.value)

IntSlider(value=10, description='m = ', max=20)

IntSlider(value=7, description='n = ', max=20, min=1)

Button(description='Aktualizovat', style=ButtonStyle())

f = 8.071408613355326 Hz
